
# Telecom X – Parte 2: Predicción de Cancelación (Churn)

**Autor/a:** _Paula Almada_  
**Fecha:** 2025-08-08

En este notebook construimos un pipeline de **Machine Learning** para predecir la probabilidad de **cancelación (churn)** en clientes de **Telecom X**.

**Objetivos:**  
- Preparar y preprocesar los datos (tratamiento, codificación, normalización).  
- Realizar análisis de correlación y selección de variables.  
- Entrenar **dos o más** modelos de clasificación.  
- Evaluar el rendimiento con métricas (Accuracy, Precision, Recall, F1, ROC-AUC).  
- Interpretar resultados (importancia de variables) y presentar **conclusiones estratégicas**.

---


## 1) 📚 Librerías y configuración

In [ ]:

# Manejo de datos
import pandas as pd
import numpy as np

# Visualización
import matplotlib.pyplot as plt
import seaborn as sns

# ML
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, ConfusionMatrixDisplay, roc_auc_score

# Estilo y opciones
pd.set_option('display.max_columns', None)
sns.set(style="whitegrid")
plt.rcParams['figure.figsize'] = (7,5)



## 2) 📥 Carga y exploración inicial

> **Opciones de carga:**
> - **A.** Subir archivo `datos_tratados.csv` al entorno de Colab y leerlo desde `/content/datos_tratados.csv`  
> - **B.** Montar Google Drive y leer desde tu carpeta

> **Nota:** Este notebook asume un archivo llamado **`datos_tratados.csv`** con una columna objetivo **`Churn`**.


In [ ]:

# === Opción A: subir archivo manualmente ===
# from google.colab import files
# uploaded = files.upload()  # luego: df = pd.read_csv('datos_tratados.csv')

# === Opción B: desde Google Drive ===
# from google.colab import drive
# drive.mount('/content/drive')
# df = pd.read_csv('/content/drive/MyDrive/ruta/datos_tratados.csv')

# === Lectura directa si ya está en /content ===
# (Asegúrate de tener 'datos_tratados.csv' en /content)
df = pd.read_csv('/content/datos_tratados.csv')

print(f"Filas: {df.shape[0]}  |  Columnas: {df.shape[1]}")
df.head()


In [ ]:

# Información general y nulos
display(df.info())
nulls = df.isna().sum().sort_values(ascending=False)
display(nulls.head(10))



## 3) 🧹 Preprocesamiento

**Pasos:**  
1. Eliminar columnas no informativas (IDs) y anidadas en forma de diccionario del EDA previo (si existieran).  
2. Separar variables numéricas y categóricas.  
3. **One-Hot Encoding** para categóricas y **escalado** para numéricas.  
4. **Train/Test split** estratificado por `Churn`.


In [ ]:

# 1) Eliminar columnas no útiles (ajusta si tu dataset cambia)
cols_to_drop = ["customerID", "customer", "phone", "internet", "account"]
df = df.drop(columns=cols_to_drop, errors='ignore')

# 2) Definir X (features) e y (target)
target_col = "Churn"
X = df.drop(columns=[target_col])
y = df[target_col]

# 3) Tipos de variables
num_cols = X.select_dtypes(include=['int64','float64']).columns.tolist()
cat_cols = X.select_dtypes(include=['object','category','bool']).columns.tolist()

print("Numéricas:", len(num_cols), "| Categóricas:", len(cat_cols))

# 4) Split estratificado
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.30, random_state=42, stratify=y
)
print("Train:", X_train.shape, " Test:", X_test.shape)
print("Proporción churn (train/test):", round(y_train.mean(),3), "/", round(y_test.mean(),3))

# 5) Preprocesamiento
preprocess = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), num_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), cat_cols)
    ],
    remainder='drop'
)



## 4) 📊 Análisis de correlación (numéricas)

> La correlación de Pearson se calcula entre variables **numéricas** y la variable objetivo `Churn`.


In [ ]:

# Matriz de correlación (solo columnas numéricas + target)
if len(num_cols) > 0:
    corr = df[num_cols + [target_col]].corr()
    # Heatmap
    plt.figure()
    sns.heatmap(corr, annot=True, fmt=".2f", cmap="coolwarm")
    plt.title("Matriz de correlación (numéricas)")
    plt.show()

    # Orden de correlación con respecto a Churn
    corr_target = corr[target_col].drop(target_col).sort_values(ascending=False)
    display(corr_target.head(10))
else:
    print("No se detectaron columnas numéricas para correlación.")



## 5) 🤖 Modelos de clasificación

Entrenamos al menos **dos modelos**:  
- **Regresión Logística** (baseline interpretable)  
- **Random Forest** (árboles en conjunto, captura relaciones no lineales)

> Métricas clave: **Accuracy, Precision, Recall, F1, ROC-AUC** y **Matriz de confusión**.


In [ ]:

# === Modelo 1: Regresión Logística ===
log_reg = Pipeline(steps=[
    ("prep", preprocess),
    ("model", LogisticRegression(max_iter=1000, n_jobs=None))
])

log_reg.fit(X_train, y_train)
pred_lr = log_reg.predict(X_test)
proba_lr = log_reg.predict_proba(X_test)[:,1]

print("=== Regresión Logística ===")
print(classification_report(y_test, pred_lr, digits=3))
print("ROC-AUC:", round(roc_auc_score(y_test, proba_lr), 3))
ConfusionMatrixDisplay.from_predictions(y_test, pred_lr)
plt.title("Matriz de confusión - Logística")
plt.show()


In [ ]:

# === Modelo 2: Random Forest ===
rf = Pipeline(steps=[
    ("prep", preprocess),
    ("model", RandomForestClassifier(
        n_estimators=300, max_depth=None, min_samples_split=2,
        min_samples_leaf=1, random_state=42, n_jobs=-1
    ))
])

rf.fit(X_train, y_train)
pred_rf = rf.predict(X_test)
proba_rf = rf.predict_proba(X_test)[:,1]

print("=== Random Forest ===")
print(classification_report(y_test, pred_rf, digits=3))
print("ROC-AUC:", round(roc_auc_score(y_test, proba_rf), 3))
ConfusionMatrixDisplay.from_predictions(y_test, pred_rf)
plt.title("Matriz de confusión - Random Forest")
plt.show()



## 6) 📈 Importancia de variables

- **Random Forest:** `feature_importances_` (importancia basada en reducción de impureza).  
- **Regresión Logística:** coeficientes para interpretar la dirección del efecto (opcional).


In [ ]:

# Extraer nombres de features transformadas
# Nota: tras fit, podemos acceder al OneHotEncoder dentro del ColumnTransformer
ohe = rf.named_steps['prep'].named_transformers_['cat']
cat_feature_names = []
if len(cat_cols) > 0:
    cat_feature_names = ohe.get_feature_names_out(cat_cols).tolist()

feature_names = num_cols + cat_feature_names
importances = rf.named_steps['model'].feature_importances_

feat_imp = pd.DataFrame({'Feature': feature_names, 'Importance': importances})
feat_imp = feat_imp.sort_values('Importance', ascending=False)

top_k = 20
display(feat_imp.head(top_k))

# Plot
plt.figure(figsize=(8,6))
sns.barplot(data=feat_imp.head(top_k), x='Importance', y='Feature')
plt.title(f"Top {top_k} variables más importantes (Random Forest)")
plt.tight_layout()
plt.show()


### Opcional: Coeficientes de la Regresión Logística (interpretabilidad)

In [ ]:

# Obtenemos el mismo orden de columnas que ve el modelo logístico
log_ohe = log_reg.named_steps['prep'].named_transformers_['cat']
log_cat_feature_names = []
if len(cat_cols) > 0:
    log_cat_feature_names = log_ohe.get_feature_names_out(cat_cols).tolist()

log_feature_names = num_cols + log_cat_feature_names
coefs = log_reg.named_steps['model'].coef_[0]
coef_df = pd.DataFrame({'Feature': log_feature_names, 'Coef': coefs}).sort_values('Coef', ascending=False)
display(coef_df.head(15))
display(coef_df.tail(15))



## 7) 📝 Conclusiones y recomendaciones

**Resumen de desempeño**
- ROC-AUC **Random Forest**: _(completar)_
- ROC-AUC **Logística**: _(completar)_
- Trade-off entre **Recall** (detectar churners) y **Precision**.

**Factores clave en churn (top importancia)**
- Variables más influyentes según RF y signos de coeficientes en Logística.

**Recomendaciones estratégicas**
- Segmentar y priorizar clientes con mayor riesgo (alto `MonthlyCharges`, contratos `Month-to-month`, sin `TechSupport`, etc. **[ajustar según resultados reales]**).
- Probar campañas de **retención** y **cross-sell** específicas.
- Monitoreo continuo con umbral de probabilidad ajustado al costo-beneficio.

---

> _Espacio para observaciones adicionales del/la analista._


### Extra (opcional): Búsqueda de hiperparámetros

In [ ]:

# from sklearn.model_selection import RandomizedSearchCV
# param_dist = {
#     'model__n_estimators': [200, 300, 500],
#     'model__max_depth': [None, 6, 10, 15],
#     'model__min_samples_split': [2, 5, 10],
#     'model__min_samples_leaf': [1, 2, 4]
# }
# rf_search = Pipeline(steps=[('prep', preprocess), ('model', RandomForestClassifier(random_state=42, n_jobs=-1))])
# rs = RandomizedSearchCV(rf_search, param_distributions=param_dist, n_iter=10, cv=3, scoring='roc_auc', n_jobs=-1, random_state=42)
# rs.fit(X_train, y_train)
# print("Mejores parámetros:", rs.best_params_)
# print("Mejor ROC-AUC (CV):", rs.best_score_)
